In [3]:
import socket
import ssl
from datetime import datetime
import pickle


class Server():
    def __init__(self, name, port, connection, priority):
        self.name = name
        self.port = port
        self.connection = connection.lower()
        self.priority = priority.lower()

        self.history = []
        self.alert = False

    def check_connection(self):
        msg = ""
        success = False
        now = datetime.now()

        try:
            if self.connection == "plain":
                socket.create_connection((self.name, self.port), timeout=10)
                msg = f"{self.name} is up. On port {self.port} with {self.connection}"
                success = True
                self.alert = False
            elif self.connection == "ssl":
                ssl.wrap_socket(socket.create_connection((self.name, self.port), timeout=10))
                msg = f"{self.name} is up. On port {self.port} with {self.connection}"
                success = True
                self.alert = False
            else:
                if self.ping():
                    msg = f"{self.name} is up. On port {self.port} with {self.connection}"
                    success = True
                    self.alert = False
        except socket.timeout:
            msg = f"server: {self.name} timeout. On port {self.port}"
        except (ConnectionRefusedError, ConnectionResetError) as e:
            msg = f"server: {self.name} {e}"
        except Exception as e:
            msg = f"No Clue??: {e}"

        
        if success == False and self.alert == False:
             #Send Alert
             self.alert = True

        self.create_history(msg,success,now)

    def create_history(self, msg, success, now):
        history_max = 100
        self.history.append((msg,success,now))

        while len(self.history) > history_max:
            self.history.pop(0)

    def ping(self):
        try:
            output = subprocess.check_output("ping -{} 1 {}".format('n' if platform.system().lower(
            ) == "windows" else 'c', self.name ), shell=True, universal_newlines=True)
            if 'unreachable' in output:
                return False
            else:
                return True
        except Exception:
                return False


if __name__ == "__main__":
    try:
        servers = pickle.load(open("servers.pickle", "rb"))
    except:
        servers = [ 
            Server("github.com", 80, "plain", "high"),
            Server("instagram.com", 80, "plain", "high"),
            Server("www.education.com", 465, "ping", "high"),
            Server("www.myntra.com", 80, "ping", "high"),
            Server("www.amazon.in", 80, "plain", "high")
        ]

    for server in servers:
        server.check_connection()
        print(len(server.history))
        print(server.history[-1])

    pickle.dump(servers, open("servers.pickle", "wb"))

3
('github.com is up. On port 80 with plain', True, datetime.datetime(2021, 6, 20, 10, 34, 41, 699945))
3
('instagram.com is up. On port 80 with plain', True, datetime.datetime(2021, 6, 20, 10, 34, 41, 743311))
3
('No Clue??: [Errno 99] Cannot assign requested address', False, datetime.datetime(2021, 6, 20, 10, 34, 41, 831165))
3
('', False, datetime.datetime(2021, 6, 20, 10, 34, 51, 858416))
3
('www.amazon.in is up. On port 80 with plain', True, datetime.datetime(2021, 6, 20, 10, 34, 51, 858465))
